**install lxml parser+ relevant modules**

In [1]:
!conda install -c anaconda lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
import numpy, pandas
import requests
import lxml
import folium
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

**lets get the name of universities in London from wikipedia (in my case, I have to use Wikizero)**

In [3]:
url="http://www.wikizeroo.org/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl91bml2ZXJzaXRpZXNfYW5kX2hpZ2hlcl9lZHVjYXRpb25fY29sbGVnZXNfaW5fTG9uZG9u"
response=requests.get(url)

In [4]:
univs=pandas.read_html(response.text)

In [5]:
univs=univs[0]
univs.head()

,University,GUG,TUG,CUG,UoL,AMBA,EQUIS,QS,THE,RG,GT,ARWU
0,"Birkbeck, University of London",113,122,115,NaN,NaN,NaN,308,301-350,NaN,NaN,NaN
1,Brunel University London,72,59,47,NaN,NaN,NaN,346,351-400,NaN,NaN,401-500
2,"City, University of London",32,75,42,NaN,NaN,NaN,343,351-400,NaN,NaN,NaN
3,"Goldsmiths, University of London",64,63,51,NaN,NaN,NaN,398,301-350,NaN,NaN,NaN
4,Imperial College London,6,4,5,a,NaN,NaN,8,8,NaN,NaN,27


**this seems it!**

In [6]:
univs=univs[["University","GUG"]]
univs

,University,GUG
0,"Birkbeck, University of London",113
1,Brunel University London,72
2,"City, University of London",32
3,"Goldsmiths, University of London",64
4,Imperial College London,6
5,King's College London,39
6,Kingston University,81
7,London Metropolitan University,117
8,London School of Economics,15
9,London South Bank University,92


**now, lets get locations for universities with the Foursquare API**

In [7]:
#we have to iterate over our "univs" Series to get each university's location.
#then we add this to our Series, making a DataFrame
lats=list()
lngs=list()

for n in univs["University"]:
    uni_params=dict(
    client_id="0M5QLNQEZR122AEXYRZKDRXOLPMNG4OENOUIFRUA4BSCJXVD",
    client_secret="NFDG24QJN34BCUD330ZFPUCU3OFJPUCHXEUQRPZR0YNI5540",
    v="20180604",
    near="51.509865, -0.118092",
    intent="checkin",
    query=n)

    url="https://api.foursquare.com/v2/venues/search"
    
    uni_response=requests.get(url, uni_params)
    uni_response=uni_response.json()
    lat, lng= uni_response["response"]["venues"][0]["location"]["lat"], uni_response["response"]["venues"][0]["location"]["lng"]
    print (n, lat, lng)
    lats.append(lat)
    lngs.append(lng)


Birkbeck, University of London 51.52203861420512 -0.13009102885223428
Brunel University London 51.5581439456925 -0.32856244493903247
City, University of London 51.515926964284816 -0.09231090545654295
Goldsmiths, University of London 51.47475738071171 -0.03637275802584594
Imperial College London 51.498805141862064 -0.17698287963867188
King's College London 51.5116270804117 -0.11637568473815918
Kingston University 51.40289411644279 -0.30323480070399456
London Metropolitan University 51.51586290011997 -0.07450332416231285
London School of Economics 51.51466435365831 -0.11660099029541014
London South Bank University 51.49758578738214 -0.10216951349047605
Middlesex University 51.613006787849216 -0.07477998733520508
Queen Mary University of London 51.52367774815883 -0.040351152420043945
Ravensbourne University London 51.52439526888408 -0.13359282839958586
Royal Holloway, University of London 51.42474136890353 -0.5667614936828613
SOAS, University of London 51.522054527013196 -0.16602668113366

**lets plot these on a map using folium**

In [8]:
univs["lats"]=lats
univs["lngs"]=lngs

In [9]:
univs.head()

,University,GUG,lats,lngs
0,"Birkbeck, University of London",113,51.522039,-0.130091
1,Brunel University London,72,51.558144,-0.328562
2,"City, University of London",32,51.515927,-0.092311
3,"Goldsmiths, University of London",64,51.474757,-0.036373
4,Imperial College London,6,51.498805,-0.176983


In [34]:
london_map=folium.Map ([51.509865, -0.118092], zoom_start=10)

In [37]:
for m in range(len(univs)):
    folium.Marker(location= [univs.lats[m],univs.lngs[m]]).add_to(london_map)

london_map

**Since our customer is interested in # of cafes near each university, lets get these from the Foursquare API! (We as for cafes within 150 mt.s to each university. We have to keep the radius short, or else we gave a significant leverage to downtown universities!)**


In [12]:
loc_counts=list()

for m in univs.index:
    uni_params=dict(
    client_id="0M5QLNQEZR122AEXYRZKDRXOLPMNG4OENOUIFRUA4BSCJXVD",
    client_secret="NFDG24QJN34BCUD330ZFPUCU3OFJPUCHXEUQRPZR0YNI5540",
    v="20180604",
    ll=f"{univs.iloc[m,2]}, {univs.iloc[m,3]}",
    intent="checkin",
    query="cafe",
    limit="50",
    radius="150")

    url="https://api.foursquare.com/v2/venues/search"

    uni_response=requests.get(url, uni_params)
    uni_response=uni_response.json()
    count=len(uni_response["response"]["venues"])
    loc_counts.append(count)
    print (univs.iloc[m,0], count)

Birkbeck, University of London 5
Brunel University London 0
City, University of London 3
Goldsmiths, University of London 6
Imperial College London 9
King's College London 7
Kingston University 1
London Metropolitan University 7
London School of Economics 8
London South Bank University 8
Middlesex University 1
Queen Mary University of London 2
Ravensbourne University London 10
Royal Holloway, University of London 1
SOAS, University of London 2
St George's, University of London 1
St Mary's University, Twickenham 0
University College London 10
University of East London 1
University of Greenwich 4
University of Roehampton 0
University of the Arts London 2
University of West London 0
University of Westminster 5


In [13]:
univs["cafes"]=loc_counts

In [14]:
univs.head()

,University,GUG,lats,lngs,cafes
0,"Birkbeck, University of London",113,51.522039,-0.130091,5
1,Brunel University London,72,51.558144,-0.328562,0
2,"City, University of London",32,51.515927,-0.092311,3
3,"Goldsmiths, University of London",64,51.474757,-0.036373,6
4,Imperial College London,6,51.498805,-0.176983,9


**and, lets add these as pop-ups to our markers on the map!**

In [15]:
for i in univs.index:
    folium.Circle(location= [univs.lats[i],univs.lngs[i]], popup=str(loc_counts[i])).add_to(london_map)

In [16]:
london_map

**lets cluster our data, to see which area has how many cafes**

In [17]:
dataset=univs[["lats","lngs","cafes"]].values
dataset

array([[ 5.15220386e+01, -1.30091029e-01,  5.00000000e+00],
       [ 5.15581439e+01, -3.28562445e-01,  0.00000000e+00],
       [ 5.15159270e+01, -9.23109055e-02,  3.00000000e+00],
       [ 5.14747574e+01, -3.63727580e-02,  6.00000000e+00],
       [ 5.14988051e+01, -1.76982880e-01,  9.00000000e+00],
       [ 5.15116271e+01, -1.16375685e-01,  7.00000000e+00],
       [ 5.14028941e+01, -3.03234801e-01,  1.00000000e+00],
       [ 5.15158629e+01, -7.45033242e-02,  7.00000000e+00],
       [ 5.15146644e+01, -1.16600990e-01,  8.00000000e+00],
       [ 5.14975858e+01, -1.02169513e-01,  8.00000000e+00],
       [ 5.16130068e+01, -7.47799873e-02,  1.00000000e+00],
       [ 5.15236777e+01, -4.03511524e-02,  2.00000000e+00],
       [ 5.15243953e+01, -1.33592828e-01,  1.00000000e+01],
       [ 5.14247414e+01, -5.66761494e-01,  1.00000000e+00],
       [ 5.15220545e+01, -1.66026681e-01,  2.00000000e+00],
       [ 5.14275849e+01, -1.74817857e-01,  1.00000000e+00],
       [ 5.14362076e+01, -3.35460535e-01

In [18]:
scaler=StandardScaler()
sc_data=scaler.fit_transform(dataset)


In [19]:
sc_data

array([[ 0.50895683,  0.23400103,  0.33257386],
       [ 1.31737501, -1.28197631, -1.14553218],
       [ 0.37211363,  0.52257564, -0.25866856],
       [-0.54969594,  0.94984606,  0.62819506],
       [-0.01125337, -0.12417136,  1.51505869],
       [ 0.27583687,  0.33876247,  0.92381627],
       [-2.15875386, -1.08851704, -0.84991097],
       [ 0.3706792 ,  0.65859467,  0.92381627],
       [ 0.34384309,  0.33704153,  1.21943748],
       [-0.03855538,  0.44727298,  1.21943748],
       [ 2.54578415,  0.65648145, -0.84991097],
       [ 0.54565794,  0.91945802, -0.55428976],
       [ 0.56172362,  0.20725336,  1.81067989],
       [-1.66958189, -3.10140385, -0.84991097],
       [ 0.50931312, -0.04048501, -0.55428976],
       [-1.60591348, -0.10763435, -0.84991097],
       [-1.41284584, -1.33466575, -1.14553218],
       [ 0.56172362,  0.20725336,  1.81067989],
       [ 0.18143278,  1.72391484, -0.84991097],
       [-0.36578544,  1.18442729,  0.03695265],
       [-0.95748501, -0.63583612, -1.145

In [20]:
means=KMeans(n_clusters=5)
means.fit(sc_data)
univs["labels"]=means.labels_
univs

,University,GUG,lats,lngs,cafes,labels
0,"Birkbeck, University of London",113,51.522039,-0.130091,5,0
1,Brunel University London,72,51.558144,-0.328562,0,2
2,"City, University of London",32,51.515927,-0.092311,3,4
3,"Goldsmiths, University of London",64,51.474757,-0.036373,6,0
4,Imperial College London,6,51.498805,-0.176983,9,0
5,King's College London,39,51.511627,-0.116376,7,0
6,Kingston University,81,51.402894,-0.303235,1,1
7,London Metropolitan University,117,51.515863,-0.074503,7,0
8,London School of Economics,15,51.514664,-0.116601,8,0
9,London South Bank University,92,51.497586,-0.102170,8,0


**label=0 has the least amount of cafes. But where is that?**

In [21]:
univs.groupby("labels").mean()

,lats,lngs,cafes
labels,,,
0,51.510490,-0.116025,7.5
1,51.442511,-0.274280,0.4
2,51.564402,-0.189790,1.0
3,51.424741,-0.566761,1.0
4,51.509597,-0.037931,2.4


In [32]:
means=univs.groupby("labels").mean()
no_cafes=folium.Map(location=[means.lats[1], means.lngs[1]], zoom_start=14)
folium.Marker(location=[means.lats[1], means.lngs[1]], popup="No Mans Land").add_to(no_cafes)

**Richmond Park!**

In [33]:
no_cafes